In [1]:
import os
ON_KAGGLE = os.path.exists('/kaggle/input')
ENABLE_SUBMISSION = False
os.environ["ON_KAGGLE"] = str(ON_KAGGLE)

In [2]:
# Install required packages if on Kaggle
if ON_KAGGLE:
    !pip install -U /kaggle/input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/hydracore120py3/hydra_core-1.2.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/omegaconf222py3/omegaconf-2.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/parglare-0-18-0/parglare-0.18.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/rank-bm25-whl/rank_bm25-0.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/amino-gaz-whl/aimo_gaz-0.0.1-py3-none-any.whl -qq --no-deps --force-reinstall
else:
    !pip install -r requirements.txt
    !pip install --upgrade build
    !python -m build
    !pip install dist/aimo_gaz-0.0.1-py3-none-any.whl --force-reinstall

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - accelerate==0.29.1
  - bitsandbytes==0.42.0
  - hatchling
  - hydra-core>=1.0.0
  - numpy==1.26.4
  - omegaconf>=2.0.1
  - parglare==0.16.1
  - peft==0.11.1
  - psutil==5.9.5
  - pydantic==1.10.13
  - rank_bm25==0.2.2
  - ray==2.4.0
  - setuptools==69.5.1
  - sympy==1.12.1
  - torch==2.1.1
  - tqdm>=4.27
  - transformers==4.39.3
  - trl==0.9.4
  - typer==0.4.0
  - urllib3>=2.0.7
* Getting build dependencies for sdist...
* Building sdist...
* Building wheel from sdist
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - accelerate==0.29.1
  - bitsandbytes==0.42.0
  - hatchling
  - hydra-core>=1.0.0
  - numpy==1.26.4
  - omegaconf>=2.0.1
  - parglare==0.16.1
  - peft==0.11.1
  - psutil==5.9.5
  - pydantic==1.10.13
  - rank_bm25==0.2.2
  - ray==2.4.0
  - setuptools==69.

In [3]:
from aimo_gaz.models.model import Model

/home/amthakur/Project/aimo/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
comet_ml is installed but `COMET_API_KEY` is not set.


In [4]:
# Set up the evaluation API
if ENABLE_SUBMISSION and ON_KAGGLE:
    import aimo

    env = aimo.make_env()
    iter_test = env.iter_test()

    # Iterate through the test set and use the model make predictions
    for test, sample_submission in iter_test:
        sample_submission['answer'] = 100 # TODO: call the model to predict the solution for text['problem']
        env.predict(sample_submission)
        print(test)
        print(sample_submission, '\n')